In [1]:
import os
import re
import json
import pytz
import string
import pandas as pd
from typing import List

---

### Dados da B3 (Dados das Empresas Listadas)

**Colunas de interesse**

- `tickers`
- `nome_pregao`
- `setor_economico`


In [2]:
tickers = ['B3SA3', 'EQTL3', 'ITUB4', 'PETR4', 'VALE3']
tickers

['B3SA3', 'EQTL3', 'ITUB4', 'PETR4', 'VALE3']

In [3]:
with open('./data/results-b3.json', encoding='utf8') as json_file:
    data = json.load(json_file)

df_b3 = pd.DataFrame(data)
df_b3.sample(5)

,razao_social,nome_pregao,segmento,tickers,setor_economico,subsetor,segmento_setorial,url_dados
428,BR MALLS PARTICIPACOES S.A.,BR MALLS PAR,NM,[BRML3],Financeiro,Exploração de Imóveis,Exploração de Imóveis,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
451,AZUL S.A.,AZUL,N2,[AZUL4],Bens Industriais,Transporte,Transporte Aéreo,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
152,OSX BRASIL S.A.,OSX BRASIL,NM,[OSXB3],Petróleo. Gás e Biocombustíveis,Petróleo. Gás e Biocombustíveis,Equipamentos e Serviços,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
326,DIBENS LEASING S.A. - ARREND.MERCANTIL,DIBENS LSG,NDA,[],Financeiro,Intermediários Financeiros,Soc. Arrendamento Mercantil,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
53,TERMOPERNAMBUCO S.A.,TERMOPE,NDA,[],Utilidade Pública,Energia Elétrica,Energia Elétrica,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...


In [4]:
df_final = pd.DataFrame()

for index, row in df_b3.iterrows():
    for ticker in row['tickers']:
        if ticker in tickers:
            df_final = df_final.append(row)

In [5]:
df_final.sort_values('razao_social')

,razao_social,nome_pregao,segmento,tickers,setor_economico,subsetor,segmento_setorial,url_dados
450,B3 S.A. - BRASIL. BOLSA. BALCÃO,B3,NM,[B3SA3],Financeiro,Serviços Financeiros Diversos,Serviços Financeiros Diversos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
296,EQUATORIAL ENERGIA S.A.,EQUATORIAL,NM,[EQTL3],Utilidade Pública,Energia Elétrica,Energia Elétrica,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
236,ITAU UNIBANCO HOLDING S.A.,ITAUUNIBANCO,N1,"[ITUB3, ITUB4, ITUB99]",Financeiro,Intermediários Financeiros,Bancos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
138,PETROLEO BRASILEIRO S.A. PETROBRAS,PETROBRAS,N2,"[PETR3, PETR4]",Petróleo. Gás e Biocombustíveis,Petróleo. Gás e Biocombustíveis,Exploração. Refino e Distribuição,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...
31,VALE S.A.,VALE,NM,[VALE3],Materiais Básicos,Mineração,Minerais Metálicos,http://bvmf.bmfbovespa.com.br/pt-br/mercados/a...


In [6]:
nomes_pregao = df_final.nome_pregao.values.tolist()
nomes_pregao

['VALE', 'PETROBRAS', 'ITAUUNIBANCO', 'EQUATORIAL', 'B3']

In [7]:
ticker_name = {
    'B3SA3': 'B3',
    'EQTL3': 'EQUATORIAL',
    'ITUB4': 'ITAUUNIBANCO',
    'PETR4': 'PETROBRAS',
    'VALE3': 'VALE'
}

---

### Dados do Portal de Notícias Suno Research

In [ ]:
with open('../crawlers/suno/results/suno-petr4.json', encoding='utf8') as json_file:
    data = json.load(json_file) 
    
df_suno = pd.DataFrame(data)

In [ ]:
# Convert date to datetime format
df_suno['date'] = pd.to_datetime(df_suno['date'], format='%d/%m/%Y %H:%M')

# Set date column as index
df_suno.set_index('date', inplace=True)

# Order by date
df_suno.sort_index(inplace=True)

In [32]:
df_suno.head()

,topic,title,search_date,url,tags
date,,,,,
2020-07-16 08:27:00,Negócios,Agenda do Dia: JHSF; Tim; Oi; Petrobras; Metal...,2022-05-03 16:22:46,https://www.suno.com.br/noticias/agenda-do-dia...,"[agenda do dia, JHSF (JHSF3), Metalfrio (FRIO3..."
2020-07-16 09:46:00,Negócios,Petrobras (PETR4) conclui venda de campos pelo...,2022-05-03 16:22:45,https://www.suno.com.br/noticias/petrobras-pet...,"[PETR3, Petrobras (PETR4)]"
2020-07-16 17:28:00,Mercado,"Ibovespa hoje cai 1,22% em mau humor com bater...",2022-05-03 16:22:45,https://www.suno.com.br/noticias/ibovespa-fech...,"[Ibovespa, JHSF (JHSF3), Petrobras (PETR4), Va..."
2020-07-16 19:04:00,Negócios,Petrobras (PETR4): exportação de petróleo para...,2022-05-03 16:22:44,https://www.suno.com.br/noticias/petrobras-pet...,"[China, PETR3, Petrobras (PETR4), Petróleo]"
2020-07-17 09:06:00,Negócios,Agenda do Dia: Moura Dubeux; Tenda; Petrobras;...,2022-05-03 16:22:44,https://www.suno.com.br/noticias/agenda-do-dia...,"[agenda do dia, Cia Hering (HGTX3), Moura Dube..."


---

### Rule-Based Method for NER

- `Suno Research`
  - Do nothing (for now) since the news portal already filter by ticker in the crawler
- `Twitter`
  - Store original tweet and mentioned ticker

#### Suno Research

In [8]:
def get_word_list(sentence: str) -> List[str]:
    list_of_words = [[w.upper().translate(str.maketrans('', '', string.punctuation))] for w in sentence.split()]
    return list_of_words

def get_tag_list(tag_list: List[str]) -> List[List[str]]:
    list_of_tags = [re.findall(r"[\w']+|[.,!?;]", x.upper()) for x in tag_list]
    return list_of_tags

In [9]:
tz = pytz.timezone('America/Sao_Paulo')

df_suno_t = pd.DataFrame()

for index, row in df_suno.iterrows():

    title = str(row['title'])
    tags = row['tags']

    # Lista de palavras, sem pontuação, com letras maiúsculas
    list_of_words = get_word_list(title)

    # Lista de tags, sem pontuação, com letras maiúsculas
    list_of_tags = get_tag_list(tags)

    # [TAGS]
    company_names_set = set()

    # Flat tag list(e.g. [['AGENDA', 'DO', 'DIA'], ['JHSF', 'JHSF3']] -> ['AGENDA', 'DO', 'DIA', 'JHSF', 'JHSF3'])
    tags_flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_tags for item in sublist]

    # 1. Verifica ocorrência exata ou do nome do pregão na lista de TAGS
    for tag_word in tags_flat_list:
        if (tag_word in tickers):
            company_names_set.add(ticker)
        if (tag_word in nomes_pregao):
            position = list(ticker_name.values()).index(tag_word)
            company_names_set.add(list(ticker_name.keys())[position])

    # Remover pontuação
    word_flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_words for item in sublist]
    
    if 'VALE' in word_flat_list:
        word_flat_list.remove('VALE')

    # [TITLE]
    # 2. Verifica ocorrência exata ou do nome do pregão no TITLE
    for title_word in word_flat_list:
        if title_word in nomes_pregao:          
            position = list(ticker_name.values()).index(title_word)
            company_names_set.add(list(ticker_name.keys())[position])

    mentioned_tickers = set(word_flat_list).intersection(tickers)
    mentioned_tickers.update(company_names_set)

    if mentioned_tickers and ('VALE3' in mentioned_tickers):
        print (f'Headline: {str(row.title)}')
        for ticker in list(mentioned_tickers):
            print (f'(datetime, ticker, topic) values ({index.replace(tzinfo=tz)}, {ticker}, {row.topic})') 
        print ('-'*35)

NameError: name 'df_suno' is not defined

---

#### Twitter

1. Get all tweets from the following list of Twitter Users:

- `@Fatosdomercado`
- `@arkoadvice`
- `@BloombergBrasil`
- `@valorinveste`
- `@InvestingBrasil`
- ...

2. Extract mentions of listed companies with Rule-Based Method for NER
3. Store into separated `pd.DataFrame` with original tweet data and mentioned ticker

In [24]:
def load_tweets(username: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Funtion to load Tweets from Specific Username (stored locally).
    """

    with open(f'../crawlers/twitter/results_related/{username}.json', encoding='utf8') as json_file:
        data = json.load(json_file) 
        
    df_twitter = pd.DataFrame(data)

    # Convert date to datetime format
    df_twitter['date'] = pd.to_datetime(df_twitter['created_at'], format='%Y-%m-%d %H:%M:%S')

    # Set date column as index
    df_twitter.set_index('date', inplace=True)

    # Order by date
    df_twitter.sort_index(inplace=True)

    # Filter Date
    df_twitter = df_twitter.loc[start_date:end_date]

    return df_twitter

In [25]:
def rule_based_ner(tweets_df: pd.DataFrame, words_to_remove: List[str]) -> pd.DataFrame:
    """
    Function to Identify Company Mentions by Word Match (Company Name or Ticker).
    """
    
    df_final = pd.DataFrame()

    for index, row in tweets_df.iterrows():
        
        title = str(row['text'])

        # Lista de palavras, sem pontuação, com letras maiúsculas
        list_of_words = [re.findall(r"[\w']+|[.,!?;]", x.upper()) for x in title.split()]

        for index, words in enumerate(list_of_words):
            if words and (words[0] in words_to_remove):
                del list_of_words[index]

        if not list_of_words:
            # Skips blank tweets
            continue 

        # Remover pontuação
        flat_list = [item.translate(str.maketrans('', '', string.punctuation)) for sublist in list_of_words for item in sublist]

        # Verifica ocorrência exata
        mentioned_tickers = set()
        mentioned_tickers = set(flat_list).intersection(tickers)
        
        for title_word in flat_list:
            if title_word in nomes_pregao:
                position = list(ticker_name.values()).index(title_word)
                company_ticker = list(ticker_name.keys())[position]
                mentioned_tickers.add(company_ticker)

        # Store all info if
        # any occurence was identified
        if mentioned_tickers:
            for ticker in list(mentioned_tickers):
                row['ticker'] = ticker
                df_final = df_final.append(row, ignore_index=True)

    return df_final

In [27]:
# Remove Sentences Starting With
remove_words = ['HTTP', 'HTTPS']

# Search Directory
directory = '../crawlers/twitter/results_related/'

df_twitter_final = pd.DataFrame()

frames = list()

# Loop over files in the directory
for filename in os.scandir(directory):
    
    username = filename.name.replace('.json', '')

    # Original Tweets
    df_twitter = load_tweets(username=username, start_date='2018-01-01', end_date='2022-07-02')

    # Original Tweets + Ticker
    df_twitter_t = rule_based_ner(
        tweets_df=df_twitter, 
        words_to_remove=remove_words
    )

    df_twitter_final = df_twitter_final.append(df_twitter_t, ignore_index=True)

In [28]:
df_twitter_final.sample(10)

,tweet_id,created_at,search_dt,text,user_id,screen_name,rt_count,favorite_count,ticker
2815,1535271857473179648,2022-06-10 11:45:09,2022-08-02 00:58:41,Petrobras: TJ-RJ suspende negociações do Polo ...,1113533606247436289,valorinveste,0.0,2.0,PETR4
701,1459500554053468164,2021-11-13 09:37:02,2022-08-02 00:49:06,Confira o Fechamento Semanal do mercado. ⬇📊\n\...,70575051,bussoladoinves,0.0,0.0,B3SA3
393,1388248227791708160,2021-04-30 18:45:44,2022-08-02 00:50:35,EXCLUSIVO. Petrobras sonda block trade de Bras...,2424278232,BrazilJournal,1.0,31.0,PETR4
1440,1536352377397288962,2022-06-13 11:18:45,2022-08-02 00:48:57,"Rápidas: Vale, siderúrgicas, Movida, Unidas, B...",70575051,bussoladoinves,0.0,1.0,VALE3
346,1313967779154927617,2020-10-07 19:21:45,2022-08-02 00:50:36,Multimesas do Itaú contrata 42 gestores em um ...,2424278232,BrazilJournal,0.0,22.0,ITUB4
1242,1507448496789544974,2022-03-25 17:05:03,2022-08-02 00:49:00,#Investir hoje para colher os resultados só de...,70575051,bussoladoinves,0.0,0.0,B3SA3
874,1474857487400644610,2021-12-25 18:40:01,2022-08-02 00:49:04,Listinha para encerrar o dia bem😁\n\nCritérios...,70575051,bussoladoinves,0.0,0.0,B3SA3
437,1488562000737275906,2022-02-01 14:16:51,2022-08-02 00:50:31,RESEARCH. Inter fica mais realista no crédito ...,2424278232,BrazilJournal,3.0,28.0,VALE3
261,1081934890277441536,2019-01-06 12:25:53,2022-08-02 00:50:43,PETROBRAS: O discurso de posse de Roberto Cas...,2424278232,BrazilJournal,4.0,20.0,PETR4
1335,1519716910350446598,2022-04-28 13:35:21,2022-08-02 00:48:59,"Atualização das 13h30: Petrobras, Vale, PetroR...",70575051,bussoladoinves,0.0,0.0,PETR4


In [29]:
df_twitter_final.ticker.value_counts(), df_twitter_final.ticker.value_counts().values.sum().sum()

(PETR4    1240
 B3SA3     908
 VALE3     598
 ITUB4     152
 EQTL3      53
 Name: ticker, dtype: int64,
 2951)

In [23]:
df_twitter_final.to_csv('../crawlers/twitter/final/df_twitter_related_users_with_mentions.csv', sep=';', encoding='utf8')